In [5]:
import torch
import kagglehub
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
import torch.optim as optim

# Download dataset
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
train_path = f'{path}/chest_xray/train/'
test_path = f'{path}/chest_xray/test/'

# Image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Datasets and DataLoaders
num_batches = 32
train_dataset = ImageFolder(root=train_path, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=num_batches, shuffle=True)

test_dataset = ImageFolder(root=test_path, transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=num_batches, shuffle=True)

# CNN Model
class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(32 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x.squeeze()

# Forward pass processing
def process_forward_pass(model, batch, criterion):
    images, labels = batch
    labels = labels.float()
    outputs = model(images)
    loss = criterion(outputs, labels)
    preds = (outputs >= 0.5).float()
    return loss, preds, labels

# Training loop
def train(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        all_preds, all_labels = [], []
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            optimizer.zero_grad()
            loss, preds, labels = process_forward_pass(model, batch, criterion)
            loss.backward()
            optimizer.step()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds)
        print(f"Epoch {epoch + 1}: Acc={accuracy * 100:.2f}%, F1={f1:.4f}")

# Testing loop
def test(model, test_loader, criterion):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            loss, preds, labels = process_forward_pass(model, batch, criterion)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    print(f"Final Test Results: Acc={accuracy * 100:.2f}%, F1={f1:.4f}")

# Initialize model, criterion, and optimizer
model = PneumoniaCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Train and test
train(model, train_loader, criterion, optimizer, epochs=5)
test(model, test_loader, criterion)


100%|██████████| 2.29G/2.29G [00:21<00:00, 115MB/s]

Extracting files...



Epoch 1/5: 100%|██████████| 163/163 [02:40<00:00,  1.02it/s]


Epoch 1: Acc=91.47%, F1=0.9434


Epoch 2/5: 100%|██████████| 163/163 [02:39<00:00,  1.02it/s]


...

Epoch 3: Acc=97.45%, F1=0.9828


Epoch 4/5: 100%|██████████| 163/163 [02:39<00:00,  1.02it/s]


Epoch 4: Acc=98.10%, F1=0.9872


Epoch 5/5: 100%|██████████| 163/163 [02:41<00:00,  1.01it/s]


Epoch 5: Acc=98.24%, F1=0.9881
Final Test Results: Acc=75.64%, F1=0.8362
